In [1]:
import sys
sys.path.append("../../../")

In [2]:
from onsager import crystal, supercell, cluster
import numpy as np
import scipy as sp
import itertools
import Transitions
import Cluster_Expansion
import MC_JIT
import time
from tqdm import tqdm
from numba import jit, int64, float64

## Set up the vector cluster expander

In [3]:
import pickle

# See if a cluster expander already exists
try:
    with open("ClusterExpander.pkl", "rb") as fl:
        print("Pre-created Cluster expander found")
        VclusExp = pickle.load(fl)

# Otherwise make one
except:
    N_units = 8 # No. of unit cells along each axis in the supercell
    NSpec = 3
    MaxOrder = 3
    MaxOrderTrans = 3

    a0 = 1.0
    cut = 1.01*a0*np.sqrt(3)/2  # Nearest neighbor cutoff

    crys = crystal.Crystal.BCC(a0, chemistry="A")
    jnetBCC = crys.jumpnetwork(0, cut)

    superlatt = N_units * np.eye(3, dtype=int)
    superBCC = supercell.ClusterSupercell(crys, superlatt)
    # Since the "spectator" tuple in the argument is left empty, all
    # sites of chemistry 0 will be mobile sites.

    vacsite = cluster.ClusterSite((0, 0), np.zeros(3, dtype=int))
    vacsiteInd = superBCC.index(np.zeros(3, dtype=int), (0, 0))[0]
    assert vacsiteInd == 0

    clusexp = cluster.makeclusters(crys, 1.01*a0, MaxOrder)
    Tclusexp = cluster.makeclusters(crys, 1.01*a0, MaxOrderTrans)

    VclusExp = Cluster_Expansion.VectorClusterExpansion(superBCC, clusexp, Tclusexp, jnetBCC,
                                                             NSpec, vacsite, MaxOrder,
                                                             MaxOrderTrans)

    print("Done setting up")
    
    # save it for later
    with open("ClusterExpander.pkl", "wb") as fl:
        pickle.dump(VclusExp, fl)

Done setting up


In [4]:
# First, we have to generate all the arrays
# Lattice gas - set all energies to zero
Energies = np.zeros(len(VclusExp.SpecClusters))
KRAEnergies = [np.zeros(len(val)) for (key, val) in VclusExp.KRAexpander.clusterSpeciesJumps.items()]

numSitesInteracts, SupSitesInteracts, SpecOnInteractSites, Interaction2En, numVecsInteracts,\
VecsInteracts, VecGroupInteracts, numInteractsSiteSpec, SiteSpecInterArray, vacSiteInd,\
InteractionIndexDict, InteractionRepClusDict, Index2InteractionDict, repClustCounter,\
Interact2RepClusArray, Interact2SymClassArray\
= VclusExp.makeJitInteractionsData(Energies)

TsInteractIndexDict, Index2TSinteractDict, numSitesTSInteracts, TSInteractSites, TSInteractSpecs, \
jumpFinSites, jumpFinSpec, FinSiteFinSpecJumpInd, numJumpPointGroups, numTSInteractsInPtGroups, \
JumpInteracts, Jump2KRAEng =\
    VclusExp.KRAexpander.makeTransJitData(KRAEnergies)

Done Indexing interactions : 1.8469595909118652
Done with chemical and symmetry class data for interactions : 1.6350266933441162
Done with vector and energy data for interactions : 2.6501846313476562


In [5]:
# Make the MC class
N_units=8
MCJit = MC_JIT.MCSamplerClass(
    numSitesInteracts, SupSitesInteracts, SpecOnInteractSites, Interaction2En, Interact2RepClusArray,
    Interact2SymClassArray, numVecsInteracts, VecsInteracts, VecGroupInteracts, numInteractsSiteSpec,
    SiteSpecInterArray, numSitesTSInteracts, TSInteractSites, TSInteractSpecs, jumpFinSites, jumpFinSpec,
    FinSiteFinSpecJumpInd, numJumpPointGroups, numTSInteractsInPtGroups, JumpInteracts, Jump2KRAEng
)
# Make the KMC class to generate trajectories
siteIndtoR, RtoSiteInd = VclusExp.makeSiteIndToSite()
KMCJit = MC_JIT.KMC_JIT(numSitesInteracts, SupSitesInteracts, SpecOnInteractSites, Interaction2En,
                        Interact2RepClusArray, Interact2SymClassArray, numVecsInteracts, VecsInteracts,
                        VecGroupInteracts, numInteractsSiteSpec, SiteSpecInterArray, numSitesTSInteracts,
                        TSInteractSites, TSInteractSpecs, jumpFinSites, jumpFinSpec, FinSiteFinSpecJumpInd,
                        numJumpPointGroups, numTSInteractsInPtGroups, JumpInteracts, Jump2KRAEng,
                        siteIndtoR, RtoSiteInd, N_units)

In [6]:
# Set up a random binary alloy
NSpec = 3
state = np.zeros(VclusExp.Nsites, dtype=int)
c0 = 0.75
N0 = int(c0*VclusExp.Nsites)
for i in range(N0):
    state[i] = 0
for i in range(N0, VclusExp.Nsites):
    state[i] = 1

# permute it
state = np.random.permutation(state)
# put the vacancy at the origin
state[vacSiteInd] = NSpec - 1
specs, counts = np.unique(state, return_counts=True)
print("N0:{}, N1:{}, Nvac:{}".format(counts[0], counts[1], counts[2]))

N0:383, N1:128, Nvac:1


In [7]:
jList, dxList = VclusExp.KRAexpander.ijList, VclusExp.KRAexpander.dxList

In [8]:
# expand
specRates = np.array([0.001, 1.0])
NVclus = len(VclusExp.vecVec)
offsc = MC_JIT.GetOffSite(state, numSitesInteracts, SupSitesInteracts, SpecOnInteractSites)

In [9]:
np.unique(offsc, return_counts=True)

(array([0, 1, 2, 3]), array([10240, 52138, 66062, 24648]))

In [28]:
WbarJit, BbarJit = MCJit.ExpandLatGas(state, jList, dxList, offsc, specRates, NVclus)

In [19]:
# Now let's do this manually
Wbar_test = np.zeros_like(WbarJit, dtype=float)
Bbar_test = np.zeros_like(BbarJit, dtype=float)

# Now test the rate expansion by explicitly constructing it
for vs1 in range(len(VclusExp.vecVec)):
    for vs2 in range(len(VclusExp.vecVec)):
        # Go through all the jumps
        for TInd in range(len(jList)):
            # For every jump, reset the offsite count
            offscjit = offsc.copy()

            vec1 = np.zeros(3, dtype=float)
            vec2 = np.zeros(3, dtype=float)

            siteB = jList[TInd]
            siteA = vacSiteInd
            # Check that the initial site is always the vacancy
            specA = state[siteA]  # the vacancy
            assert specA==NSpec - 1
            specB = state[siteB]

            for interactnun in range(numInteractsSiteSpec[siteA, specA]):
                interactInd = SiteSpecInterArray[siteA, specA, interactnun]
                repClus = InteractionRepClusDict[Index2InteractionDict[interactInd]]
                vecList = VclusExp.clust2vecClus[repClus]

                if len(vecList) > 0:
                    assert numVecsInteracts[interactInd]==len(vecList)
                else:
                    assert numVecsInteracts[interactInd]==-1

                for i in range(numVecsInteracts[interactInd]):
                    assert np.allclose(VecsInteracts[interactInd, i, :],
                                       VclusExp.vecVec[vecList[i][0]][vecList[i][1]])
                    
                if offscjit[interactInd] == 0:
                    for tupInd, tup in enumerate(vecList):
                        if tup[0] == vs1:
                            vec1 -= VecsInteracts[interactInd, tupInd, :]
                    for tupInd, tup in enumerate(vecList):
                        if tup[0] == vs2:
                            vec2 -= VecsInteracts[interactInd, tupInd, :]
                offscjit[interactInd] += 1

            for interactnun in range(numInteractsSiteSpec[siteB, specB]):
                interactInd = SiteSpecInterArray[siteB, specB, interactnun]
                repClus = InteractionRepClusDict[Index2InteractionDict[interactInd]]
                vecList = VclusExp.clust2vecClus[repClus]

                if len(vecList) > 0:
                    assert numVecsInteracts[interactInd]==len(vecList)
                else:
                    assert numVecsInteracts[interactInd]==-1

                for i in range(numVecsInteracts[interactInd]):
                    assert np.allclose(VecsInteracts[interactInd, i, :],
                                       VclusExp.vecVec[vecList[i][0]][vecList[i][1]])
                    
                if offscjit[interactInd] == 0:
                    for tupInd, tup in enumerate(vecList):
                        if tup[0] == vs1:
                            vec1 -= VecsInteracts[interactInd, tupInd, :]
                    for tupInd, tup in enumerate(vecList):
                        if tup[0] == vs2:
                            vec2 -= VecsInteracts[interactInd, tupInd, :]
                offscjit[interactInd] += 1

            for interactnun in range(numInteractsSiteSpec[siteA, specB]):
                interactInd = SiteSpecInterArray[siteA, specB, interactnun]

                repClus = InteractionRepClusDict[Index2InteractionDict[interactInd]]
                vecList = VclusExp.clust2vecClus[repClus]

                if len(vecList) > 0:
                    assert numVecsInteracts[interactInd]==len(vecList)
                else:
                    assert numVecsInteracts[interactInd]==-1

                for i in range(numVecsInteracts[interactInd]):
                    assert np.allclose(VecsInteracts[interactInd, i, :],
                                       VclusExp.vecVec[vecList[i][0]][vecList[i][1]])

                offscjit[interactInd] -= 1
                if offscjit[interactInd] == 0:
                    for tupInd, tup in enumerate(vecList):
                        if tup[0] == vs1:
                            vec1 += VecsInteracts[interactInd, tupInd, :]
                    for tupInd, tup in enumerate(vecList):
                        if tup[0] == vs2:
                            vec2 += VecsInteracts[interactInd, tupInd, :]

            for interactnun in range(numInteractsSiteSpec[siteB, specA]):
                interactInd = SiteSpecInterArray[siteB, specA, interactnun]

                repClus = InteractionRepClusDict[Index2InteractionDict[interactInd]]
                vecList = VclusExp.clust2vecClus[repClus]

                if len(vecList) > 0:
                    assert numVecsInteracts[interactInd]==len(vecList)
                else:
                    assert numVecsInteracts[interactInd]==-1

                for i in range(numVecsInteracts[interactInd]):
                    assert np.allclose(VecsInteracts[interactInd, i, :],
                                       VclusExp.vecVec[vecList[i][0]][vecList[i][1]])

                offscjit[interactInd] -= 1
                if offscjit[interactInd] == 0:
                    for tupInd, tup in enumerate(vecList):
                        if tup[0] == vs1:
                            # assertEqual(VecGroupInteracts[interactInd, tupInd], vs1)
                            vec1 += VecsInteracts[interactInd, tupInd, :]
                    for tupInd, tup in enumerate(vecList):
                        if tup[0] == vs2:
                            # assertEqual(VecGroupInteracts[interactInd, tupInd], vs2)
                            vec2 += VecsInteracts[interactInd, tupInd, :]
            # get the rate
            rate = specRates[specB]
            # get the dot product
            dot = np.dot(vec1, vec2)

            Wbar_test[vs1, vs2] += rate*dot
            if vs1 == 0:
                Bbar_test[vs2] += rate*np.dot(dxList[TInd], vec2)

        assert np.allclose(WbarJit[vs1, vs2], Wbar_test[vs1, vs2])

assert np.allclose(BbarJit, Bbar_test)
print("Test Complete - Wbar and Bbar matched against JIT calculations explicitly")

Test Complete - Wbar and Bbar matched against JIT calculations explicitly


In [11]:
%%timeit
offsc = MC_JIT.GetOffSite(state, numSitesInteracts, SupSitesInteracts, SpecOnInteractSites)

4.31 ms ± 113 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
lamb = np.zeros(3)
for interactInd in range(offsc.shape[0]):
    